# Your Title Here

**Name(s)**: Atherv Vidhate & Zubin Sannakkayala

**Website Link**: (your website link)

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

import plotly.express as px
pd.options.plotting.backend = 'plotly'

# from dsc80_utils import * # Feel free to uncomment and use this.

## Step 1: Introduction

In [3]:
# TODO


## Step 2: Data Cleaning and Exploratory Data Analysis

In [90]:
# TODO
outages = pd.read_csv('outage.csv').replace('NA', np.nan).set_index('OBS')

# Convert start dates and times into pd.Timestamp objects, as well as combine columns into a condensed START and RESTORATION columns
outages['OUTAGE.START.DATE'] = pd.to_datetime(outages['OUTAGE.START.DATE'] , errors="coerce")
outages['OUTAGE.START.TIME'] = pd.to_datetime(outages['OUTAGE.START.TIME']).dt.time
outages['OUTAGE.RESTORATION.DATE'] = pd.to_datetime(outages['OUTAGE.RESTORATION.DATE'] , errors="coerce")
outages['OUTAGE.RESTORATION.TIME'] = pd.to_datetime(outages['OUTAGE.RESTORATION.TIME']).dt.time
outages['OUTAGE.START'] = pd.to_datetime(outages['OUTAGE.START.DATE'].astype(str) + ' ' + outages['OUTAGE.START.TIME'].astype(str), errors='coerce')
outages['OUTAGE.RESTORATION'] = pd.to_datetime(outages['OUTAGE.RESTORATION.DATE'].astype(str) + ' ' + outages['OUTAGE.RESTORATION.TIME'].astype(str), errors='coerce')
outages['is_day'] = (outages['OUTAGE.START'].dt.hour >= 8) & (outages['OUTAGE.START'].dt.hour < 16)
outages['Hour'] = outages['OUTAGE.START'].dt.hour

def get_daypart(hour):
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    elif (0 <= hour < 5) | (21 <= hour < 24):
        return 'Night'
    else:
        return np.nan

outages['Daypart'] = outages['Hour'].apply(get_daypart)


filtered = outages[['U.S._STATE', 'NERC.REGION', 'CLIMATE.REGION', 'ANOMALY.LEVEL', 'OUTAGE.START', 'OUTAGE.RESTORATION', 'CAUSE.CATEGORY',
         'CAUSE.CATEGORY.DETAIL', 'HURRICANE.NAMES', 'OUTAGE.DURATION', 'CUSTOMERS.AFFECTED', 'RES.PERCEN', 'COM.PERCEN', 'IND.PERCEN', 'RES.CUST.PCT',
         'PC.REALGSP.STATE', 'POPDEN_URBAN', 'PCT_WATER_TOT', 'Hour', 'Daypart']]



/var/folders/cj/mk6_35t16dg3wd3vjv0ymvw00000gn/T/ipykernel_2346/379959789.py:6: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/var/folders/cj/mk6_35t16dg3wd3vjv0ymvw00000gn/T/ipykernel_2346/379959789.py:8: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



In [79]:
# Univariate analysis, looking at the distribution of Outage Durations over time
fig = px.histogram(outages, x='OUTAGE.DURATION', histnorm='probability')

fig.update_layout(
    title={
        'text': "Distribution of Outage Durations",
        'x': 0.5,  
        'xanchor': 'center', 
        'yanchor': 'top'
    },
    xaxis_title='Outage Duration (minutes)',   
    yaxis_title='Probability',
    width=700, 
    height=500)

fig.show()
fig.to_html(full_html=False, include_plotlyjs='cdn')
fig.write_html("output.html", full_html=False, include_plotlyjs='cdn')

In [80]:
# Univariate Analysis - NERC Region distribution breakdown
#
region_counts = outages['NERC.REGION'].value_counts().reset_index()
region_counts.columns = ['Region', 'Count']

fig_2 = px.pie(
    region_counts,
    names='Region',  
    values='Count',  
    title='NERC Region Distribution',
    color_discrete_sequence=px.colors.qualitative.Set2,
    width=700,
    height=500,
)

fig_2.show()

In [81]:
# Bivariate Analysis - Outage Duration vs Customers Affected
fig_3 = px.scatter(
    outages,
    x="OUTAGE.DURATION",  
    y="CUSTOMERS.AFFECTED",  
    title="Outage Duration vs Customers Affected",
    labels={"OUTAGE.DURATION": "Outage Duration (minutes)", "CUSTOMERS.AFFECTED": "Customers Affected"},
    template="plotly_white",  
    size_max=10,
    width=700,
    height=500
)

# Show the plot
fig_3.show()

In [82]:
# Bivariate Analysis - visualize the summed outage durations by state, more populous states have more outages
state_outage_duration = outages.groupby('U.S._STATE')['OUTAGE.DURATION'].sum().reset_index()

fig_4 = px.bar(
    state_outage_duration.sort_values('OUTAGE.DURATION', ascending=False),
    x='U.S._STATE',  
    y='OUTAGE.DURATION',  
    title="Total Outage Duration by State",
    labels={'U.S._STATE': 'U.S. State', 'OUTAGE.DURATION': 'Outage Duration (minutes)'},
    template="plotly_white",  
    color='OUTAGE.DURATION',  
    color_continuous_scale='Viridis'  
)

fig_4.show()

In [83]:
# Interesting Aggregate Analysis - Pivot Table for Outage Duration broken down by the part of day
pivot_table = outages.pivot_table(
    values='OUTAGE.DURATION',   
    index='Daypart',            
    aggfunc='mean',             
    fill_value=0               
)

pivot_table

,OUTAGE.DURATION
Daypart,
Afternoon,1980.546584
Evening,2779.808271
Morning,3000.726437
Night,2992.256849


## Step 3: Assessment of Missingness

In [88]:
# TODO
filtered

def tvd_of_groups(df, groups, cats) :
    cnts = df-pivot_table(index=cats, columns=groups, aggfunc='size')
    # Normalize each column.
    distr = cnts / cnts. sum()
    # Compute and return the TVD.
    return distr.diff(axis=1). iloc[:, -1].abs().sum() / 2


,U.S._STATE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,OUTAGE.START,OUTAGE.RESTORATION,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,HURRICANE.NAMES,OUTAGE.DURATION,CUSTOMERS.AFFECTED,RES.PERCEN,COM.PERCEN,IND.PERCEN,RES.CUST.PCT,PC.REALGSP.STATE,POPDEN_URBAN,PCT_WATER_TOT,Hour,Daypart
OBS,,,,,,,,,,,,,,,,,,,,
1,Minnesota,MRO,East North Central,-0.3,2011-07-01 17:00:00,2011-07-03 20:00:00,severe weather,NaN,NaN,3060.0,70000.0,35.549073,32.225029,32.202431,88.9448,51268,2279.0,8.407334,17.0,Evening
2,Minnesota,MRO,East North Central,-0.1,2014-05-11 18:38:00,2014-05-11 18:39:00,intentional attack,vandalism,NaN,1.0,NaN,30.032487,34.210389,35.727564,88.8335,53499,2279.0,8.407334,18.0,Evening
3,Minnesota,MRO,East North Central,-1.5,2010-10-26 20:00:00,2010-10-28 22:00:00,severe weather,heavy wind,NaN,3000.0,70000.0,28.097672,34.501015,37.365983,88.9206,50447,2279.0,8.407334,20.0,Evening
4,Minnesota,MRO,East North Central,-0.1,2012-06-19 04:30:00,2012-06-20 23:00:00,severe weather,thunderstorm,NaN,2550.0,68200.0,31.994099,33.543330,34.439329,88.8954,51598,2279.0,8.407334,4.0,Night
5,Minnesota,MRO,East North Central,1.2,2015-07-18 02:00:00,2015-07-19 07:00:00,severe weather,NaN,NaN,1740.0,250000.0,33.982576,36.205850,29.779498,88.8216,54431,2279.0,8.407334,2.0,Night
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1530,North Dakota,MRO,West North Central,-0.9,2011-12-06 08:00:00,2011-12-06 20:00:00,public appeal,NaN,NaN,720.0,34500.0,37.212544,33.351628,29.435904,83.8598,57012,2192.2,2.401765,8.0,Morning
1531,North Dakota,MRO,West North Central,NaN,NaT,NaT,fuel supply emergency,Coal,NaN,NaN,NaN,NaN,NaN,NaN,84.6901,42913,2192.2,2.401765,NaN,NaN
1532,South Dakota,RFC,West North Central,0.5,2009-08-29 22:54:00,2009-08-29 23:53:00,islanding,NaN,NaN,59.0,NaN,36.564432,40.124517,23.311051,84.1773,45230,2038.3,1.692256,22.0,Night


In [92]:
# TODO
n_repetitions = 10000
shuffled = outages.copy() 
observed_diff = shuffled.groupby('is_day')['CUSTOMERS.AFFECTED'].mean()
obs = observed_diff[False] - observed_diff[True]
mean_diffs = []
for _ in range(n_repetitions):
    shuffled['is_day'] = np.random.permutation(shuffled['is_day'])
    calc = shuffled.groupby('is_day')['CUSTOMERS.AFFECTED'].mean()
    mean_diffs.append(calc[False] - calc[True])

p_value = (obs < mean_diffs).mean()

fig = px.histogram(mean_diffs, x=0, nbins=100, histnorm='probability', 
                   title='Empirical Distribution of the Difference in Group Means')
fig.add_vline(x=obs, line_color='red', line_width=2, opacity=1)
fig.add_annotation(text=f'<span style="color:red">Observed Difference = {round(obs, 2)}</span>',
                   x=obs, showarrow=False, y=0.04)
fig.update_layout(yaxis_range=[0, 0.05], xaxis_title = 'Simulated Differences', yaxis_title = 'Probability',
                  title={
        'text': "Distribution of Outage Durations",
        'x': 0.5,
        'xanchor': 'center', 
        'yanchor': 'top'
    })


## Step 5: Framing a Prediction Problem

In [93]:
# TODO
filtered['CLIMATE.REGION'].value_counts()

CLIMATE.REGION
Northeast             350
South                 229
West                  217
Central               200
Southeast             153
East North Central    138
Northwest             132
Southwest              92
West North Central     17
Name: count, dtype: int64

## Step 6: Baseline Model

In [ ]:
# TODO

## Step 7: Final Model

In [ ]:
# TODO

## Step 8: Fairness Analysis

In [ ]:
# TODO